In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt
import datetime
from sklearn.cross_validation import train_test_split

In [2]:
DATA = pd.concat(pd.read_csv('dis_acct_e.csv', chunksize=1000, date_parser=pd.datetools.to_datetime))
TARGETData = 'bal_outst_amt'

In [3]:
DATA.head()

,me_dt,acct_id,acct_sub_type,acct_opnd_dt,acct_opng_brnch_id,acct_sts,cust_id,prod_id,fincg_type_cd,tenor,...,inlien_ind,cury_cd,bal_outst_amt,avg_bal_out_globl_amt,avg_bal_out_globl_amt_cr,avg_bal_out_globl_amt_dr,fund_cr_amt,fund_chrg_amt,inlien_amt,no_of_time_renewed
0,2015-12-31,18124905,CURRENT ACCOUNT/OVERDRAFT,2013-07-18,3426827,CLOSE,11716809,6302,191,NaN,...,N,MYR,16.289508,15.649508,15.969508,24.839508,15.679508,7.129508,16.259508,0
1,2015-12-31,17816466,CURRENT ACCOUNT/OVERDRAFT,2013-04-10,3426532,CLOSE,11503652,6302,191,NaN,...,N,MYR,73659.291778,44321.011778,58990.151778,78314.921778,44385.081778,39729.451778,73595.221778,0
2,2015-12-31,2189400,CURRENT ACCOUNT/OVERDRAFT,2013-11-27,3426268,ACTIVE,116097,5831,111,NaN,...,N,MYR,172067.371356,84835.531356,128451.451356,158898.531356,84934.011356,98102.851356,171968.891356,0
3,2015-12-31,17851353,CURRENT ACCOUNT/OVERDRAFT,2014-07-20,3426565,ACTIVE,12568229,6302,191,NaN,...,N,MYR,1969.166978,1094.726978,1531.946978,1977.716978,1096.206978,1087.656978,1967.686978,0
4,2015-12-31,18208198,CURRENT ACCOUNT/OVERDRAFT,2014-01-22,3426892,DORMANT,12123857,6302,191,NaN,...,N,MYR,44622.457314,24656.457314,34639.457314,44622.457314,24689.557314,24689.557314,44589.357314,0


In [4]:
print(DATA.columns)

Index([u'me_dt', u'acct_id', u'acct_sub_type', u'acct_opnd_dt',
       u'acct_opng_brnch_id', u'acct_sts', u'cust_id', u'prod_id',
       u'fincg_type_cd', u'tenor', u'tenor_uom', u'dep_maturity_dt',
       u'cert_iss_dt', u'remain_term', u'remain_term_uom',
       u'maturity_handle_codes', u'joint_acct_ind', u'new_acct_ind',
       u'inlien_ind', u'cury_cd', u'bal_outst_amt', u'avg_bal_out_globl_amt',
       u'avg_bal_out_globl_amt_cr', u'avg_bal_out_globl_amt_dr',
       u'fund_cr_amt', u'fund_chrg_amt', u'inlien_amt', u'no_of_time_renewed'],
      dtype='object')


In [6]:
print(isinstance(DATA['acct_opnd_dt'].iloc[1], datetime.date))
print(DATA['acct_opnd_dt'].iloc[1])

False
2013-04-10


In [7]:
dateColumns = ['me_dt', 'acct_opnd_dt', 'cert_iss_dt', 'dep_maturity_dt']
for x in dateColumns:
    DATA[x] = pd.to_datetime(DATA[x])
    print(x, "is", DATA['me_dt'].dtype)

('me_dt', 'is', dtype('<M8[ns]'))
('acct_opnd_dt', 'is', dtype('<M8[ns]'))
('cert_iss_dt', 'is', dtype('<M8[ns]'))
('dep_maturity_dt', 'is', dtype('<M8[ns]'))


In [8]:
DATA_clean = DATA.set_index('acct_id').select_dtypes(exclude=['datetime64[ns]', np.flexible], include=[np.number])

In [9]:
DATA_clean.head()

,acct_opng_brnch_id,cust_id,prod_id,fincg_type_cd,tenor,remain_term,bal_outst_amt,avg_bal_out_globl_amt,avg_bal_out_globl_amt_cr,avg_bal_out_globl_amt_dr,fund_cr_amt,fund_chrg_amt,inlien_amt,no_of_time_renewed
acct_id,,,,,,,,,,,,,,
18124905,3426827,11716809,6302,191,NaN,0,16.289508,15.649508,15.969508,24.839508,15.679508,7.129508,16.259508,0
17816466,3426532,11503652,6302,191,NaN,0,73659.291778,44321.011778,58990.151778,78314.921778,44385.081778,39729.451778,73595.221778,0
2189400,3426268,116097,5831,111,NaN,0,172067.371356,84835.531356,128451.451356,158898.531356,84934.011356,98102.851356,171968.891356,0
17851353,3426565,12568229,6302,191,NaN,0,1969.166978,1094.726978,1531.946978,1977.716978,1096.206978,1087.656978,1967.686978,0
18208198,3426892,12123857,6302,191,NaN,0,44622.457314,24656.457314,34639.457314,44622.457314,24689.557314,24689.557314,44589.357314,0


In [10]:
def cleanNulls(data):
    for x in data:
        sumNull = data[x].isnull().sum()
        if len(data) / sumNull > 0.5:
            print("dropping ", x)
            data = data.drop(x, axis=1)
    return data

DATA_clean = cleanNulls(DATA_clean)

('dropping ', 'tenor')


In [11]:
def checkNulls(data):
    for x in data:
        print(x)
        print(data[x].isnull().sum(), len(data) / data[x].isnull().sum())

checkNulls(DATA_clean)

acct_opng_brnch_id
(0, 0)
cust_id
(0, 0)
prod_id
(0, 0)
fincg_type_cd
(0, 0)
remain_term
(0, 0)
bal_outst_amt
(0, 0)
avg_bal_out_globl_amt
(0, 0)
avg_bal_out_globl_amt_cr
(0, 0)
avg_bal_out_globl_amt_dr
(0, 0)
fund_cr_amt
(0, 0)
fund_chrg_amt
(0, 0)
inlien_amt
(0, 0)
no_of_time_renewed
(0, 0)


In [12]:
DATA_noInt = DATA.set_index('acct_id').select_dtypes(exclude=['floating', 'int', 'datetime64[ns]'])
DATA_noInt.head()

,acct_sub_type,acct_sts,tenor_uom,remain_term_uom,maturity_handle_codes,joint_acct_ind,new_acct_ind,inlien_ind,cury_cd
acct_id,,,,,,,,,
18124905,CURRENT ACCOUNT/OVERDRAFT,CLOSE,NaN,NaN,NaN,N,N,N,MYR
17816466,CURRENT ACCOUNT/OVERDRAFT,CLOSE,NaN,NaN,NaN,N,N,N,MYR
2189400,CURRENT ACCOUNT/OVERDRAFT,ACTIVE,NaN,NaN,NaN,N,N,N,MYR
17851353,CURRENT ACCOUNT/OVERDRAFT,ACTIVE,NaN,NaN,NaN,N,N,N,MYR
18208198,CURRENT ACCOUNT/OVERDRAFT,DORMANT,NaN,NaN,NaN,N,N,N,MYR


In [13]:
checkNulls(DATA_noInt)

acct_sub_type
(0, 0)
acct_sts
(0, 0)
tenor_uom
(1006236, 1)
remain_term_uom
(1006236, 1)
maturity_handle_codes
(1084324, 1)
joint_acct_ind
(0, 0)
new_acct_ind
(0, 0)
inlien_ind
(0, 0)
cury_cd
(0, 0)


In [15]:
DATA_noInt = cleanNulls(DATA_noInt)

('dropping ', 'tenor_uom')
('dropping ', 'remain_term_uom')
('dropping ', 'maturity_handle_codes')


In [16]:
DATA_noInt.iloc[:,0:].apply(pd.Series.value_counts)

,acct_sub_type,acct_sts,joint_acct_ind,new_acct_ind,inlien_ind,cury_cd
ACTIVE,NaN,1008399.0,NaN,NaN,NaN,NaN
CLOSE,NaN,100953.0,NaN,NaN,NaN,NaN
CURRENT ACCOUNT/OVERDRAFT,1006236.0,NaN,NaN,NaN,NaN,NaN
DORMANT,NaN,35042.0,NaN,NaN,NaN,NaN
FIXED DEPOSIT - CERTIFICATE,138158.0,NaN,NaN,NaN,NaN,NaN
MYR,NaN,NaN,NaN,NaN,NaN,1144394.0
N,NaN,NaN,1144274.0,1128424.0,1126376.0,NaN
Y,NaN,NaN,120.0,15970.0,18018.0,NaN


In [17]:
DATA_noInt_Vec = pd.get_dummies(DATA_noInt).astype('float32')

In [18]:
print(len(DATA_noInt_Vec))
DATA_noInt_Vec.head()

1144394


,acct_sub_type_CURRENT ACCOUNT/OVERDRAFT,acct_sub_type_FIXED DEPOSIT - CERTIFICATE,acct_sts_ACTIVE,acct_sts_CLOSE,acct_sts_DORMANT,joint_acct_ind_N,joint_acct_ind_Y,new_acct_ind_N,new_acct_ind_Y,inlien_ind_N,inlien_ind_Y,cury_cd_MYR
acct_id,,,,,,,,,,,,
18124905,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
17816466,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
2189400,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
17851353,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
18208198,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0


In [16]:
# print(dis_acct_e.columns.difference(dis_acct_e_nonInt.columns))
# dis_acct_e_int1 = dis_acct_e.ix[:, dis_acct_e.columns.difference(dis_acct_e_nonInt.columns)]
# dis_acct_e_int1.head()

In [17]:
# dis_acct_e_int2 = dis_acct_e_int1.set_index('acct_id')
# dis_acct_e_nonInt_vec.head()
# pd.merge(dis_acct_e_int2, dis_acct_e_nonInt_vec, on=dis_acct_e_int2.index)

In [19]:
DATA_clean_Vec = DATA_clean.join(DATA_noInt_Vec)

In [20]:
DATA_clean_Vec.columns

Index([u'acct_opng_brnch_id', u'cust_id', u'prod_id', u'fincg_type_cd',
       u'remain_term', u'bal_outst_amt', u'avg_bal_out_globl_amt',
       u'avg_bal_out_globl_amt_cr', u'avg_bal_out_globl_amt_dr',
       u'fund_cr_amt', u'fund_chrg_amt', u'inlien_amt', u'no_of_time_renewed',
       u'acct_sub_type_CURRENT ACCOUNT/OVERDRAFT',
       u'acct_sub_type_FIXED DEPOSIT - CERTIFICATE', u'acct_sts_ACTIVE',
       u'acct_sts_CLOSE', u'acct_sts_DORMANT', u'joint_acct_ind_N',
       u'joint_acct_ind_Y', u'new_acct_ind_N', u'new_acct_ind_Y',
       u'inlien_ind_N', u'inlien_ind_Y', u'cury_cd_MYR'],
      dtype='object')

In [20]:
DATA_clean_Vec

,acct_opng_brnch_id,cust_id,prod_id,fincg_type_cd,remain_term,bal_outst_amt,avg_bal_out_globl_amt,avg_bal_out_globl_amt_cr,avg_bal_out_globl_amt_dr,fund_cr_amt,...,remain_term_uom_MONTHS,maturity_handle_codes_AUTO RENEW,maturity_handle_codes_NO AUTO RENEW,joint_acct_ind_N,joint_acct_ind_Y,new_acct_ind_N,new_acct_ind_Y,inlien_ind_N,inlien_ind_Y,cury_cd_MYR
acct_id,,,,,,,,,,,,,,,,,,,,,
938277,3426102,11746706,5831,111,0,1.910189e+01,17.201892,1.815189e+01,2.765189e+01,17.231892,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
938277,3426102,11746706,5831,111,0,1.910189e+01,17.201892,1.815189e+01,2.765189e+01,17.231892,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
938277,3426102,11746706,5831,111,0,1.910189e+01,17.201892,1.815189e+01,2.765189e+01,17.231892,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
938277,3426102,11746706,5831,111,0,1.910189e+01,17.201892,1.815189e+01,2.765189e+01,17.231892,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
938277,3426102,11746706,5831,111,0,1.910189e+01,17.201892,1.815189e+01,2.765189e+01,17.231892,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
938277,3426102,11746706,5831,111,0,1.910189e+01,17.201892,1.815189e+01,2.765189e+01,17.231892,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
938277,3426102,11746706,5831,111,0,4.240897e+00,2.340897,3.290897e+00,4.240897e+00,2.340897,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
938277,3426102,11746706,5831,111,0,4.240897e+00,2.340897,3.290897e+00,4.240897e+00,2.340897,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
938277,3426102,11746706,5831,111,0,4.240897e+00,2.340897,3.290897e+00,4.240897e+00,2.340897,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0


In [21]:
y = DATA_clean_Vec.pop(TARGETData)
X = DATA_clean_Vec.head(1000000)

In [22]:
X_train,X_test,y_train,y_test = train_test_split(X,y.head(1000000),test_size=0.2)

In [23]:
# iX.loc[X_train] # return dataframe train
print(len(X_train))
X_train

800000


,acct_opng_brnch_id,cust_id,prod_id,fincg_type_cd,remain_term,avg_bal_out_globl_amt,avg_bal_out_globl_amt_cr,avg_bal_out_globl_amt_dr,fund_cr_amt,fund_chrg_amt,...,acct_sts_ACTIVE,acct_sts_CLOSE,acct_sts_DORMANT,joint_acct_ind_N,joint_acct_ind_Y,new_acct_ind_N,new_acct_ind_Y,inlien_ind_N,inlien_ind_Y,cury_cd_MYR
acct_id,,,,,,,,,,,,,,,,,,,,,
17189895,3426207,13400573,5831,111,0,1.024346e+03,1.439126e+03,1.853906e+03,1.025666e+03,1.025666e+03,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
2997660,3426514,12522228,6302,191,0,3.666126e+04,5.294351e+04,6.718101e+04,3.670846e+04,3.875321e+04,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
17327796,3426302,13573511,5831,111,0,2.190912e+04,2.887933e+04,3.855192e+04,2.193981e+04,1.923743e+04,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
17112062,3426140,13744628,5831,111,0,8.359500e+03,1.174450e+04,1.512950e+04,8.370190e+03,8.370190e+03,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
2202933,3426410,118200,5831,111,0,2.433002e+02,3.418202e+02,4.403402e+02,2.436102e+02,2.436102e+02,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
17188251,3426204,13181509,5831,111,0,4.957971e+03,7.081351e+03,9.040151e+03,4.964211e+03,5.128791e+03,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
17316694,3426298,13077710,5831,111,0,2.656439e+04,3.891397e+04,4.899938e+04,2.659639e+04,2.886056e+04,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
9380689,3426858,12188716,6302,191,0,1.488640e+04,1.620623e+04,2.421762e+04,1.491185e+04,8.220288e+03,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
9380844,3426861,12843678,5463,191,0,5.806968e+05,1.362800e+06,1.363661e+06,5.772114e+05,1.361036e+06,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0


In [27]:
print(len(X_train))
print(len(X_test))
print(len(y_train))
print(len(y_test))

800000
200000
800000
200000


In [30]:
X_train.to_csv("./trainTestDatasets/X_train_dis_acct_e.csv")

In [32]:
X_train.to_csv("./trainTestDatasets/X_train_dis_acct_e.csv")
X_test.to_csv("./trainTestDatasets/X_test_dis_acct_e.csv")
y_train.to_csv("./trainTestDatasets/y_train_dis_acct_e.csv")
y_test.to_csv("./trainTestDatasets/y_test_dis_acct_e.csv")

AttributeError: 'Series' object has no attribute 'tocsv'

In [33]:
y_test.to_csv("./trainTestDatasets/y_test_dis_acct_e.csv")